In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HBox, VBox, Label, Layout

import re
import csv
import yaml
import urllib.parse

from kgutils import create_widget_tuple_list
from kgutils import read_wd_selector_list

creator_options = []
dropdown_options = []
csv_dict = {}
query_dict = {}

KG_CONFIG_FILE = 'kg-config.yml'
csv_template = 'kg-{}.csv'

institutions_file = csv_template.format('P195')

### SPARQL templates 

sparql_top='''
# Knowledge Graph code generated from wikidata-kg
#defaultView:Graph
SELECT ?item1 ?image1 ?item1Label ?item2 ?image2 ?item2Label ?size ?rgb 
WHERE 
{
'''

sparql_union_template='''
 { # Cluster facet%s
  VALUES ?rgb { "fff033" }  # Color definition
  VALUES ?size { 1 }
  # Filters
%s
  # Cluster
%s
  OPTIONAL { ?item1 wdt:P18 ?image1. }
 }
'''

sparql_bottom='''
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''

sparql_facets_values_template  = '  VALUES ?facet%s { %s }  # %s\n'
sparql_facets_filter_template  = '  ?item1 wdt:%s ?facet%s .\n'
sparql_facets_cluster_template = '  ?item1 wdt:%s ?item2 .\n'
sparql_facets_any_template     = '  ?item1 wdt:%s [] .\n'

### Functions

def gen_query(params: list) -> str:
    '''
    Given a list of tuples, each with property, qids, and filter type, generate a SPARQL query

    Parameters
    ----------
    params: list of three-part tuples like [('P31', ('wd:Q3305213', 'wd:Q93184', 'wd:Q860861'), 'filter') ...]
    '''
    _sparql_facets_values = ''
    _sparql_facets_filter = ''
    
    # First pass: create VALUES statements and filters
    _facet_n = 1 # Counter for facets, start at 1
    for i in params:
        # Handle VALUES statement ie. VALUES ?facet1 {wd:Q160236}  # P195
        _wd_list = ' '.join(i[1])
        # Handle all 'filter+cluster' or 'filter' directives
        if i[0] and i[2] != 'cluster':
            # Need to check if it is 'filter+cluster' so to add an ?item1 wdt:P31 ?item2 . statement

            # Create block of SPARQL code at top that 
            # If "Any" is selected, don't put in VALUES statement
            if i[1] and (i[1][0] != 'Any'):
                _sparql_facets_values += sparql_facets_values_template % (int(_facet_n), _wd_list, str(i[0]))

            # Create block of SPARQL code that filters based on facet choices
            # Check in case there is a null tuple coming in. TODO: improve this in the future.
            if i[1]:
                # If "Any" is selected
                if i[1][0] == 'Any':
                    # put in a blank node selector []
                    _sparql_facets_filter += sparql_facets_any_template % (str(i[0]))
                else:
                    # regular case of facets
                    _sparql_facets_filter += sparql_facets_filter_template % (str(i[0]), int(_facet_n))

        _facet_n += 1

    # Start to assemble the SPARQL query
    _sparql_string = sparql_top
    _sparql_string += _sparql_facets_values

    # Use second pass for making any potential cluster statements (UNION)
    _facet_n = 1
    _sparql_union_statements = ''

    for i in params:
        _union_block = ''
        # Handle all cluster directives
        if i[0] and (i[2] == 'cluster' or i[2] == 'filter+cluster'):
            # Add clustering statement with the property
            _sparql_facets_cluster = sparql_facets_cluster_template % i[0]
            # Create UNION block
            _union_block = sparql_union_template % (_facet_n, _sparql_facets_filter, _sparql_facets_cluster)

            if _sparql_union_statements:  # Add UNION to connect the statements
                _sparql_union_statements += '\n  UNION\n'
            _sparql_union_statements += _union_block

            _facet_n += 1

    # Finish the SPARQL code
    _sparql_string += _sparql_union_statements
    _sparql_string += sparql_bottom

    return _sparql_string

def live_mode() -> bool:
    """
    Returns whether we are in live mode or not, by checking widget
    """
    return (livetoggle_button.value == 'On')

def handle_livetoggle(incoming: dict):
    """
    Handler for when the live mode is toggled on or off
    """
    draw_kg()
    return

def handle_filter(incoming: dict):
    """
    Handle a click on a filter radio button
    """
    _boxindex = int(incoming['owner'].description.split()[1])-1
    _selectionbox = topbox.children[_boxindex].children[2]

    if incoming['new'] == 'cluster':
        _selectionbox.disabled = True
    else:
        _selectionbox.disabled = False

    draw_kg()
    return

def handle_selector(incoming: dict):
    """
    Handle a click on a selector
    """
    _boxindex = int(incoming['owner'].description.split()[1])-1

    if 'Any' in incoming['new']:
        query_dict[_boxindex] = []
    else:
        query_dict[_boxindex] = incoming['new']

    draw_kg()
    return

def handle_dropdown(incoming: dict):
    """
    Handle when dropdown menu selection happens
    
    incoming dict looks like this: 
    {'name': 'value', 
     'old': 'Artist/creator (P131)', 
     'new': 'Institution (P195)', 
     'owner': 
        Dropdown(description='Facet 2', index=2, 
                 options=('-', 'Instance (P31)', 'Institution (P195)', 'Artist/creator (P131)', 'Depiction (P180)'), 
                 value='Institution (P195)'), 
     'type': 'change'}
    """
    # Calculate the index into Hbox that contains this widget
    # ie. convert "Dropdown(description='Facet 2'..." into the number 1
    _boxindex = int(incoming['owner'].description.split()[1])-1

    # Determine the right selection box to populate with options
    # Selection box is the third (index=2) widget, after the pulldown and filter
    _selectionbox = topbox.children[_boxindex].children[2]

    if incoming['new'] == '-':
        _selectionbox.options = []
    else:
        pnumber = title_to_predicate(incoming['new'])
        _selectionbox.options = csv_dict[pnumber]
    draw_kg()
    return

def add_box(incoming: dict):
    """
    Add a new box to the interface, consisting of a dropdown, filter and selector
    These are named Facet 1, Filter 1, Param 1... and so on as the numbers are unique
    """

    # Set up unique names for widgets based on sequence number
    _sequence = len(topbox.children)+1
    _facet = 'Facet {}'.format(_sequence)
    _param = 'Param {}'.format(_sequence)
    _filter = 'Filter {}'.format(_sequence)

    _new_dropdown = widgets.Dropdown(
        options=dropdown_options,
        value='-',
        description=_facet,
        disabled=False,
    )
    _new_list = widgets.SelectMultiple(
        options=[],
        value=[],
        rows=10,
        description=_param,
        disabled=False
    )
    _new_filter = widgets.ToggleButtons(
        options=['filter',  'cluster', 'filter+cluster'],
        value='filter+cluster', # Default
        # TODO: better sizing - layout={'width': 'max-content'}, # If the items' names are long
        description=_filter,
        style={"button_width": "30%"},
        layout=Layout(flex='1 1 auto', width='auto'),
        disabled=False
    )
    
    _new_dropdown.observe(handle_dropdown, names='value')
    _new_list.observe(handle_selector, names='value')
    _new_filter.observe(handle_filter, names='value')

    _new_selector = VBox([_new_dropdown,_new_filter,_new_list])

    # Add new selector to the topbox, could also use += here instead of splat (Python 3)
    topbox.children = (*topbox.children, _new_selector)

    return

def remove_box(incoming: dict):
    """
    Remove a box from the inteface by simply pruning it from the list
    """
    topbox.children = topbox.children[:len(topbox.children)-1]
    draw_kg()
    return

def reset_boxes(incoming: dict):
    """
    Reset the dropdowns of all selectors to -, thereby triggering the blanking of the selector
    """
    for i in topbox.children:
        if (i.children[0].value != '-'):
            i.children[0].value = '-'
    draw_kg(mode='blank')
    return

def title_to_predicate(title: str) -> str:
    '''
    Given 'Institution (P195)' return 'P195'
    '''
    try:
        _result = re.search('\((P[0-9]+)\)$', title)
        return _result.group(1)
    except AttributeError:
        return None

def title_to_csvfilename(title: str) -> str:
    '''
    Given 'Institution (P195)' return 'kg-P195.csv'
    '''
    try:
        _result = title_to_predicate(title)
        return csv_template.format(_result)
    except AttributeError:
        return None

def gather_predicates() -> list:
    '''
    Return a list of all Wikidata properties associated with dropdown menus currently selected
    
    Sample output: ['P195', 'P31', None, 'P131']
    '''
    _returnlist = []
    for _selector in topbox.children:
        # Extract string currently in the dropdown menu
        dropstring = _selector.children[0].value
        # Grab just the P number in parens
        _pnumber = title_to_predicate(_selector.children[0].value)
        # Append to list
        _returnlist += [_pnumber]
    return _returnlist

def gather_filters() -> list:
    '''
    Return a list of all filter settings

    Sample output: ['filter', 'cluster', 'filter+cluster']
    '''
    _returnlist = []
    for _selector in topbox.children:
        # Extract string currently in the dropdown menu
        _selected_string = _selector.children[1].value
        # Append to list
        _returnlist += [_selected_string]
    return _returnlist

def gather_selections() -> list:
    '''
    Return a list of all selected items in a selection box
    
    Sample output: ['boy', 'man', None, 'tree']
    '''
    _returnlist = []
    for _selector in topbox.children:
        # Extract string currently in the selection box
        _selected_string = _selector.children[2].value
        # Append to list
        _returnlist += [_selected_string]
    return _returnlist


def draw_kg(mode='normal'):

    _query = None
    _htmlcode = ''
    
    if mode == 'normal':
        if not live_mode():
            return

        _predicates = gather_predicates()
        _selections = gather_selections()
        _filters    = gather_filters()

        # TODO: put in toggle on whether to show these results in interface or HTML code
        #     _htmlcode += '<pre>Predicates: {}</pre>'.format(_predicates)
        #     _htmlcode += '<pre>Selections: {}</pre>'.format(_selections)
        #     _htmlcode += '<pre>Filters: {}</pre>'.format(_filters)

        # Create a list of all the params
        _params = list(zip(_predicates, _selections, _filters))

        _query = gen_query(_params)

    if _query:
        _graphurl = 'https://query.wikidata.org/embed.html#' + urllib.parse.quote(_query)
    else:
        _graphurl = 'about:blank'

    # Load iframe
    _iframecode = '<iframe src=' + _graphurl + ' width=900 height=600></iframe>'
    _htmlcode += _iframecode

    graphoutput.value = _htmlcode

### Execution

# Read yaml config
with open(KG_CONFIG_FILE, 'r') as stream:
    try:
        options = yaml.safe_load(stream)
        dropdown_options = options['dropdown_options']
    except yaml.YAMLError as exc:
        print(exc)
        raise
        
# Should load each property CSV file if they exist, store it in csv_dict
for p in dropdown_options[1:]:
    prop = title_to_predicate(p)
    csv_dict[prop] = read_wd_selector_list(title_to_csvfilename(p))
    # print (title_to_csvfilename(p), len(csv_dict[prop]))

### Create interface

textheader = widgets.HTML(
    value="<H1>Wikidata Graph Browser</H1><P>Browse Wikidata knowledge graphs by specifying facets.</P>",
    placeholder='',
    description='',
)

livetoggle_button = widgets.ToggleButtons(
    options=['On', 'Off'],
    description='Live view:',
    value='Off',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Do live Wikidata queries every click', 'Don\'t do live queries'],
)

graphoutput = widgets.HTML(
    value=u'',
    placeholder='<p>Waiting for input</p>',
    description=''
)

livetoggle_button.observe(handle_livetoggle, names='value')

add_button    = widgets.Button(value='', description='Add')
remove_button = widgets.Button(value='', description='Remove')
reset_button  = widgets.Button(value='', description='Reset')

topbox = HBox()
    
add_button.on_click(add_box)
remove_button.on_click(remove_box)
reset_button.on_click(reset_boxes)

buttonbar = HBox([add_button, remove_button, reset_button,livetoggle_button])
add_box(None)
add_box(None)

headerbox = VBox([textheader,buttonbar,topbox])
bottombox = HBox([graphoutput])

bigbox = VBox([headerbox,bottombox])

display(bigbox)

### End execution, start event handling